In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''

  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
file = open("/content/sample_data/matter.txt", "r")
FileContent = file.read().strip()

In [ ]:
FileContent

'"Global warming" redirects here. For other uses, see Climate change (disambiguation) and Global warming (disambiguation). This article is about contemporary climate change. For historical climate trends, see Climate variability and change.\nThe global map shows sea temperature rises of 0.5 to 1 degree Celsius; land temperature rises of 1 to 2 degree Celsius; and Arctic temperature rises of up to 4 degrees Celsius.\nChanges in surface air temperature over the past 50 years.[1] The Arctic has warmed the most, and temperatures on land have generally increased more than sea surface temperatures.\n\nEarth\'s average surface air temperature has increased almost 1.5 °C (about 2.5 °F) since the Industrial Revolution. Natural forces cause some variability, but the 20-year average shows the progressive influence of human activity.[2]\nIn common usage, climate change describes global warming—the ongoing increase in global average temperature—and its effects on Earth\'s climate system. Climate ch

In [ ]:
len(FileContent)


21345

In [ ]:

# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [ ]:
tokenizer.model_max_length

1024

In [ ]:
tokenizer.max_len_single_sentence

1022

In [ ]:
 tokenizer.num_special_tokens_to_add()

2

In [ ]:
pip install nltk


In [ ]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

96

In [ ]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

5

In [ ]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1002, 1014, 994, 1014, 542]

In [ ]:
[len(tokenizer(c).input_ids) for c in chunks]

[1004, 1016, 996, 1016, 544]

In [ ]:
sum([len(tokenizer(c).input_ids) for c in chunks])

4576

In [ ]:
len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (4468 > 1024). Running this sequence through the model will result in indexing errors


4468

In [ ]:
len(tokenizer.tokenize(FileContent))

4466

In [ ]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 Earth's average surface air temperature has increased almost 1.5 °C since the Industrial Revolution. Carbon dioxide, the primary greenhouse gas driving global warming, has grown by about 50% and is at levels unseen for millions of years. 2023 was the warmest on record at +1.48 °C (2.66 °F) since regular tracking began in 1850.
 Global warming became more popular term after NASA climate scientist James Hansen used it in 1988 testimony in the U.S. Senate. Climate change can also be used more broadly to include changes to the climate that have happened throughout Earth's history. The 2013–2022 decade warmed to an average 1.15 °C [1.00–1.25 °C] compared to the pre-industrial baseline.
 The thermal energy in the global climate system has grown with only brief pauses since at least 1970. Over 90% of this extra energy has been stored in the ocean. The Northern Hemisphere and the North Pole have warmed much faster than the South Pole and Southern Hemisphere. The IPCC expects the 20-year avera